In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import metrics

In [3]:
df = pd.read_csv('Twitter_stock_final_dataset_new.csv')
df.head()

,Year,Month,Day,StockName,Positive,Negative,Neutral,Total Tweets,Close,Volume,Open,High,Low,Day_of_week
0,2020,1,1,apple,10,2,8,20,75.0875,135647440.0,74.0600,75.1500,73.7975,Wednesday
1,2020,1,1,microsoft,9,0,11,20,160.6200,22634550.0,158.7800,160.7300,158.3300,Wednesday
2,2020,1,1,tesla,17,3,3,23,86.0520,47791930.0,84.9000,86.1391,84.3420,Wednesday
3,2020,1,1,nvidia,1,0,0,1,59.9775,23767876.0,59.6875,59.9775,59.1800,Wednesday
4,2020,1,1,paypal,1,0,1,2,110.7500,7419283.0,109.4700,111.2100,109.2750,Wednesday


In [4]:
df2 = df

In [5]:
Date = ['Year', 'Month','Day']
df2.sort_values(by = Date, inplace = True, ascending = True)

In [6]:
# seperate independent vs dependent vriables

#cols = ['Close', 'High', 'Low', 'Volume','Open','Day_of_week']
X_reg3 = df2.drop('Close', axis = 1).values
y_reg3 = df2.Close.values

In [7]:
df_X = pd.DataFrame(X_reg3)#, columns=['Column1', 'Column2'] 
df_X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,2020,1,1,apple,10,2,8,20,135647440.0,74.06,75.15,73.7975,Wednesday
1,2020,1,1,microsoft,9,0,11,20,22634550.0,158.78,160.73,158.33,Wednesday
2,2020,1,1,tesla,17,3,3,23,47791930.0,84.9,86.1391,84.342,Wednesday
3,2020,1,1,nvidia,1,0,0,1,23767876.0,59.6875,59.9775,59.18,Wednesday
4,2020,1,1,paypal,1,0,1,2,7419283.0,109.47,111.21,109.275,Wednesday


In [8]:
#LABEL ENCODE
# Encoding the Independent or ohject variable Day_of_week
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

regEnc = LabelEncoder()

X_reg3[:, -1] = regEnc.fit_transform(X_reg3[:, -1]) #Day_of_week

X_reg3[:, 3] = regEnc.fit_transform(X_reg3[:, 3]) #stockName

In [9]:
#Split the new data into training and testing sets

Xtrain_set3, Xtest_set3= np.split(X_reg3, [int(0.8 * len(X_reg3))])
ytrain_set3, ytest_set3= np.split(y_reg3, [int(0.8 * len(y_reg3))])

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestRegressor()

params = {'max_depth':[5,10,30,50],
         'min_samples_split':[2,3,5,10],
         'criterion':['mse','mae']}


cross_val = RandomizedSearchCV(estimator = rf, 
                          param_distributions = params,
                          cv = 5,
                          n_iter = 10,
                          n_jobs = -1,
                          verbose = 10)

cross_val.fit(Xtrain_set3,ytrain_set3)
cross_val.best_params_

print('the best parameters are : ', cross_val.best_params_)

rf_best = cross_val.best_estimator_
rf_best.fit(Xtrain_set3,ytrain_set3)

y_Lpred3 = rf_best.predict(Xtest_set3)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
the best parameters are :  {'min_samples_split': 3, 'max_depth': 10, 'criterion': 'mae'}


In [13]:
#Scores

print('RANDOM FOREST SCORES----')
print()

def mape(actual, predicted):
    mape = np.mean(np.abs((actual - predicted)/actual))*100
    return mape

#mean absolute percentage error
print('mean absolute percentage error: ', mape(ytest_set3, y_Lpred3))

#rmse
mse =np.square(np.subtract(ytest_set3, y_Lpred3)).mean()
rmse = math.sqrt(mse)
print('rmse: ', rmse)

#Accuracy
print('Mean absolute error: ', metrics.mean_absolute_error(ytest_set3, y_Lpred3))

#r2_score
print('The r2_score: ', metrics.r2_score(ytest_set3, y_Lpred3))

RANDOM FOREST SCORES----

mean absolute percentage error:  0.7287244740317284
rmse:  3.956924906953294
Mean absolute error:  2.5254312885209367
The r2_score:  0.9995675540386167


In [12]:
result = pd.DataFrame({'Actual': ytest_set3.flatten(), 'Predicted': y_Lpred3.flatten()})
result

,Actual,Predicted
0,126.543333,126.780054
1,248.910000,249.001937
2,597.920000,586.390633
3,154.052500,154.760172
4,254.355000,250.860352
...,...,...
592,730.170000,733.520300
593,211.130000,211.054867
594,269.910000,268.999170
595,212.460000,211.689108
